### Importaciones

In [1]:
import os
import io
import uuid
import fitz  # PyMuPDF
from fuzzywuzzy import fuzz
from dotenv import load_dotenv
from agents import Agent, OpenAIChatCompletionsModel, trace, Runner, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from IPython.display import display
from office365.runtime.auth.authentication_context import AuthenticationContext
from office365.sharepoint.client_context import ClientContext
from office365.sharepoint.files.file import File
from azure.core.credentials import AzureKeyCredential
from azure.ai.documentintelligence import DocumentIntelligenceClient
from openai import AsyncAzureOpenAI

In [2]:
load_dotenv(override=True)

True

### Variables de entorno

In [ ]:
azure_openai_key = os.getenv('AZURE_OPENAI_KEY')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')

azure_openai_gpt_deployment = os.getenv('AZURE_OPENAI_API_DEPLOYMENT')
azure_openai_gpt_version = os.getenv('AZURE_OPENAI_API_VERSION')
azure_openai_gpt_model = os.getenv('AZURE_OPENAI_API_MODEL')

azure_document_intelligence_key = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_KEY")
azure_document_intelligence_endpoint = os.getenv("AZURE_DOCUMENT_INTELLIGENCE_ENDPOINT")

sharepoint_user = os.getenv('SHAREPOINT_USER')
sharepoint_password = os.getenv('SHAREPOINT_PASSWORD')
sharepoint_site_url = os.getenv('SHAREPOINT_SITE_URL')
sharepoint_dossier_url = os.getenv('SHAREPOINT_DOSSIER_URL')

### Funciones

In [ ]:
def download_file_from_sharepoint(ruta_relativa: str) -> bytes:
    """
    Descarga directamente un archivo desde SharePoint usando su ruta relativa completa.
    """

    print(f"📥 [download_file_from_sharepoint] Descargando desde ruta completa: '{ruta_relativa}'")

    # Autenticación
    ctx_auth = AuthenticationContext(sharepoint_site_url)
    if not ctx_auth.acquire_token_for_user(sharepoint_user, sharepoint_password):
        raise Exception("Error al autenticarse en SharePoint.")

    ctx = ClientContext(sharepoint_site_url, ctx_auth)

    # Ruta completa del archivo en SharePoint
    file_url = f"{sharepoint_dossier_url}/{ruta_relativa}"
    print(f"🔗 [download_file_from_sharepoint] URL completa: {file_url}")

    response = File.open_binary(ctx, file_url)
    print("✅ [download_file_from_sharepoint] Archivo descargado exitosamente")
    return response.content

In [5]:
def analyze_document_content(file_bytes: bytes) -> str:
    """
    Analiza un documento PDF usando Azure Document Intelligence (prebuilt-read) y guarda el resultado como .md.

    Args:
        file_bytes (bytes): Contenido del archivo PDF.

    Returns:
        str: Contenido OCR del documento.
    """

    print(f"📥 [analyze_document_content] Procesando archivo OCR ({len(file_bytes)} bytes)")

    client = DocumentIntelligenceClient(
        endpoint=azure_document_intelligence_endpoint,
        credential=AzureKeyCredential(azure_document_intelligence_key)
    )

    stream = io.BytesIO(file_bytes)

    poller = client.begin_analyze_document(
        model_id="prebuilt-read",
        body=stream,
        content_type="application/pdf"
    )

    result = poller.result()
    full_text = result.content

    # Guardar en archivo Markdown
    os.makedirs("output", exist_ok=True)
    output_filename = f"ocr_result_{uuid.uuid4().hex[:8]}.md"
    output_path = os.path.join("output", output_filename)
    with open(output_path, "w", encoding="utf-8") as f:
        f.write(full_text)

    print(f"✅ [analyze_document_content] Resultado guardado en: {output_path}")
    return full_text

In [6]:
def es_pdf_con_texto(file_bytes: bytes) -> bool:
    """
    Determina si un PDF contiene texto digital o si es solo una imagen escaneada.
    Si contiene texto, lo guarda en un archivo .md en la carpeta 'output'.

    Args:
        file_bytes (bytes): Contenido del archivo PDF.

    Returns:
        bool: True si el PDF contiene texto, False si solo contiene imágenes.
    """
    print("🕵️ [es_pdf_con_texto] Analizando si el PDF contiene texto embebido...")

    try:
        with fitz.open(stream=file_bytes, filetype="pdf") as doc:
            texto_extraido = ""
            for page_num, page in enumerate(doc, start=1):
                texto = page.get_text().strip()
                if texto:
                    print(f"✅ [es_pdf_con_texto] Texto encontrado en página {page_num}")
                    texto_extraido += texto + "\n"

            if texto_extraido.strip():
                # Guardar el texto en un archivo Markdown
                os.makedirs("output", exist_ok=True)
                output_filename = f"texto_pdf_{uuid.uuid4().hex[:8]}.md"
                output_path = os.path.join("output", output_filename)
                with open(output_path, "w", encoding="utf-8") as f:
                    f.write(texto_extraido)

                print(f"📝 [es_pdf_con_texto] Texto guardado en: {output_path}")
                return True

            print("📷 [es_pdf_con_texto] No se encontró texto embebido. El PDF parece ser una imagen escaneada.")
            return False

    except Exception as e:
        print(f"❌ [es_pdf_con_texto] Error al procesar el PDF: {e}")
        return False

In [7]:
def contiene_palabra_clave(document_content: str, palabra_clave: str) -> bool:
    """
    Verifica si una palabra o frase está presente en el contenido del documento.

    Args:
        document_content (str): Texto plano del documento (result.content).
        palabra_clave (str): Palabra o frase a buscar.

    Returns:
        bool: True si se encuentra, False si no.
    """

    print(f"🔍 [contiene_palabra_clave] Buscando palabra: '{palabra_clave}'")

    if not document_content or not palabra_clave:
        print("⚠️ [contiene_palabra_clave] Texto o palabra vacíos")
        return False

    found = palabra_clave.lower() in document_content.lower()
    print(f"✅ [contiene_palabra_clave] ¿Palabra encontrada?: {found}")
    return found


### Tools

In [ ]:
@function_tool
def verificar_palabra_en_documento(ruta_relativa: str, palabra_clave: str) -> bool:
    """
    Ejecuta el flujo completo: descarga el archivo desde SharePoint usando una ruta relativa completa,
    analiza su contenido (OCR solo si es necesario) y verifica si contiene una palabra.

    Args:
        ruta_relativa (str): Ruta relativa del archivo en SharePoint (ej. 'M1/1. PoA/PoA Uruguay.pdf').
        palabra_clave (str): Palabra o frase a buscar en el documento.

    Returns:
        bool: True si se encuentra la palabra, False si no.
    """

    print(f"🚀 [verificar_palabra_en_documento] Iniciando flujo para ruta relativa '{ruta_relativa}'")

    try:
        # Paso 1: Descargar archivo desde SharePoint
        print(f"⬇️ [verificar_palabra_en_documento] Descargando archivo desde SharePoint...")
        file_bytes = download_file_from_sharepoint(ruta_relativa)
        print(f"📄 [verificar_palabra_en_documento] Archivo descargado: {len(file_bytes)} bytes")

        # Paso 2: Verificar si el PDF tiene texto embebido
        if es_pdf_con_texto(file_bytes):
            print("🧾 [verificar_palabra_en_documento] Usando texto embebido del PDF.")
            with fitz.open(stream=file_bytes, filetype="pdf") as doc:
                contenido_texto = "\n".join(page.get_text() for page in doc)
        else:
            print("📷 [verificar_palabra_en_documento] Usando OCR de Azure Document Intelligence.")
            contenido_texto = analyze_document_content(file_bytes)

        # Paso 3: Verificar si contiene la palabra clave
        resultado = contiene_palabra_clave(document_content=contenido_texto, palabra_clave=palabra_clave)
        print(f"🔍 [verificar_palabra_en_documento] ¿Contiene la palabra '{palabra_clave}'?: {resultado}")
        return resultado

    except Exception as e:
        print(f"❌ [verificar_palabra_en_documento] Error en el proceso: {e}")
        return False

In [9]:
def cargar_rutas_desde_md(archivo_md: str) -> list[str]:
    rutas = []
    with open(archivo_md, "r", encoding="utf-8") as f:
        for linea in f:
            linea = linea.strip()
            if linea.startswith("- "):
                rutas.append(linea[2:])
    return rutas

def buscar_con_fuzzy(query: str, rutas: list[str], threshold: int = 50) -> list[tuple[str, int]]:
    puntuaciones = [(ruta, fuzz.token_set_ratio(query, ruta)) for ruta in rutas]
    return sorted(
        [(ruta, score) for ruta, score in puntuaciones if score >= threshold],
        key=lambda x: x[1],
        reverse=True
    )

class RutaPunteada(BaseModel):
    ruta: str = Field(description="Ruta relativa del documento.")
    puntaje: int = Field(description="Puntaje de similitud con la palabra clave.")

@function_tool
def buscar_rutas_relativas(consulta: str) -> list[RutaPunteada]:
    """
    Devuelve las rutas relativas ordenadas por relevancia junto con su puntaje.
    """
    archivo_md = "output/dossier_estructura.md"

    print(f"🔎 [buscar_rutas_relativas] Iniciando búsqueda con palabra clave: '{consulta}'")
    rutas = cargar_rutas_desde_md(archivo_md)
    print(f"📂 Total de rutas cargadas: {len(rutas)}")

    substr_matches = [(r, 100) for r in rutas if consulta.lower() in r.lower()]
    fuzzy_matches = buscar_con_fuzzy(consulta, rutas, threshold=50)

    todo = substr_matches + fuzzy_matches
    todo_unico = {}
    for ruta, score in todo:
        if ruta not in todo_unico or score > todo_unico[ruta]:
            todo_unico[ruta] = score

    resultados_ordenados = sorted(todo_unico.items(), key=lambda x: x[1], reverse=True)

    print(f"✅ Rutas encontradas: {len(resultados_ordenados)}")
    if resultados_ordenados:
        print(f"🏅 Mejor coincidencia: Ruta='{resultados_ordenados[0][0]}', Puntaje={resultados_ordenados[0][1]}")

    return [RutaPunteada(ruta=ruta, puntaje=score) for ruta, score in resultados_ordenados]

In [ ]:
# Instrucciones para el agente
ruta_instructions = (
    "Sos un agente experto en identificar documentos relevantes dentro de una estructura de directorios.\n"
    "Tu tarea es buscar todas las rutas relativas dentro de un archivo Markdown que contengan documentos relacionados "
    "con una palabra clave provista por el usuario.\n"
    "Debes utilizar la función 'buscar_rutas_relativas' para obtener todas las coincidencias posibles.\n"
    "El archivo donde buscar se pasa como parámetro ('archivo_md'), y la palabra clave como 'consulta'.\n"
    "Retorná la lista de rutas en formato JSON."
)

class ItemRutas(BaseModel):
    ruta: str = Field(description="Ruta relativa del documento.")
    puntaje: int = Field(description="Puntaje de similitud de la ruta con la palabra clave.")

class ListaRutas(BaseModel):
    rutas: list[ItemRutas] = Field(description="Lista de rutas relativas de documentos que coinciden con una palabra clave.")

client_buscador = AsyncAzureOpenAI(
    api_key=azure_openai_key,
    api_version=azure_openai_gpt_version,
    azure_endpoint=azure_openai_endpoint
)

# Crear agente y convertirlo en herramienta
buscador_rutas = Agent(
    name="Buscador de Rutas",
    output_type=ListaRutas,
    instructions=ruta_instructions,
    model=OpenAIChatCompletionsModel(
                model=azure_openai_gpt_deployment,
                openai_client=client_buscador,
            ),
    tools=[buscar_rutas_relativas] 
)

rutas_tool = buscador_rutas.as_tool(
    tool_name="obtener_rutas_documentos",
    tool_description="Devuelve una lista de rutas relativas de documentos que coinciden con una palabra clave."
)

In [ ]:
# Instrucciones para el agente
verificar_instructions = (
    "Sos un agente que debe verificar si un documento contiene una palabra clave determinada.\n"
    "Para eso, debés usar la herramienta 'verificar_palabra_en_documento'.\n"
    "El documento puede contener texto embebido o requerir OCR, y la verificación debe hacerse sobre el contenido completo.\n"
    "Retorná solo True o False según si la palabra fue encontrada o no."
)

# Schema de salida
class ResultadoVerificacion(BaseModel):
    value: bool = Field(description="True si se encontró la palabra en el documento, False si no.")

# Cliente OpenAI (podés compartir el mismo `client_buscador`)
client_verificador = AsyncAzureOpenAI(
    api_key=azure_openai_key,
    api_version=azure_openai_gpt_version,
    azure_endpoint=azure_openai_endpoint
)

# Agente como herramienta
verificador_palabra = Agent(
    name="Verificador de Palabra",
    instructions=verificar_instructions,
    output_type=ResultadoVerificacion,
    model=OpenAIChatCompletionsModel(
        model=azure_openai_gpt_deployment,
        openai_client=client_verificador,
    ),
    tools=[verificar_palabra_en_documento]
)

verificar_tool = verificador_palabra.as_tool(
    tool_name="verificar_si_documento_contiene_palabra",
    tool_description="Verifica si un documento descargado desde SharePoint contiene una palabra clave específica."
)

In [ ]:
INSTRUCTIONS = """
Sos un asistente experto en búsqueda documental.

Tu objetivo es determinar si una palabra específica aparece en documentos relacionados con una consulta dada. Para ello:

1. Usá `obtener_rutas_documentos` con la palabra clave para obtener la lista de documentos más relevantes.
2. Elegí **solo el primer resultado** (el más relevante), accediendo al campo `ruta`.
3. Separá esa ruta en `subfolder_name` y `file_name`. El `file_name` es la última parte después del `/`, y el resto es el `subfolder_name`.
4. Usá `verificar_palabra_en_documento` pasando estos valores junto con la palabra clave original.
5. Si se encuentra la palabra, respondé exactamente: `Sí, la palabra fue encontrada`
6. Si no se encuentra, respondé exactamente: `No, no se encontró la palabra`

No expliques el resultado. No uses rutas intermedias. No adivines.
"""

client = AsyncAzureOpenAI(
    api_key=azure_openai_key,
    api_version=azure_openai_gpt_version,
    azure_endpoint=azure_openai_endpoint
)

search_agent = Agent(
    name="Verificador de Palabra Clave",
    instructions=INSTRUCTIONS,
    tools=[rutas_tool, verificar_tool],
    model=OpenAIChatCompletionsModel(
                model=azure_openai_gpt_deployment,
                openai_client=client,
            ),
    model_settings=ModelSettings(tool_choice="required")
)

In [ ]:
message = """
Buscar documentos relacionados con la palabra 'PoA'.
Verificar si en alguno de ellos aparece la palabra 'ADCETRIS'.
"""

with trace("Search"):  
    result = await Runner.run(search_agent, message)
    print(result.final_output)

OPENAI_API_KEY is not set, skipping trace export


🔎 [buscar_rutas_relativas] Iniciando búsqueda con palabra clave: 'PoA'
📂 Total de rutas cargadas: 158
✅ Rutas encontradas: 1
🏅 Mejor coincidencia: Ruta='M1/1. PoA/PoA Uruguay.pdf', Puntaje=100


OPENAI_API_KEY is not set, skipping trace export


🚀 [verificar_palabra_en_documento] Iniciando flujo para ruta relativa 'M1/1. PoA/PoA Uruguay.pdf'
⬇️ [verificar_palabra_en_documento] Descargando archivo desde SharePoint...
📥 [SharePoint] Descargando desde ruta completa: 'M1/1. PoA/PoA Uruguay.pdf'
🔗 [SharePoint] URL completa: /sites/LaboratorioLibra/Documentos compartidos/PoC IA Libra Dossier/Dossier/M1/1. PoA/PoA Uruguay.pdf


OPENAI_API_KEY is not set, skipping trace export


✅ [SharePoint] Archivo descargado exitosamente
📄 [verificar_palabra_en_documento] Archivo descargado: 136651 bytes
🕵️ [es_pdf_con_texto] Analizando si el PDF contiene texto embebido...
📷 [es_pdf_con_texto] No se encontró texto embebido. El PDF parece ser una imagen escaneada.
📷 [verificar_palabra_en_documento] Usando OCR de Azure Document Intelligence.
📥 [analyze_document_content] Procesando archivo OCR (136651 bytes)
✅ [analyze_document_content] Resultado guardado en: output\ocr_result_b5a3984e.md
🔍 [contiene_palabra_clave] Buscando palabra: 'ADCETRIS'
✅ [contiene_palabra_clave] ¿Palabra encontrada?: True
🔍 [verificar_palabra_en_documento] ¿Contiene la palabra 'ADCETRIS'?: True


OPENAI_API_KEY is not set, skipping trace export


Sí, la palabra fue encontrada


OPENAI_API_KEY is not set, skipping trace export
